In [3]:
import pandas as pd
import numpy as np

In [4]:
import torch

### 파일

In [5]:
df = pd.read_csv('42.csv')
df.head()

,title,content,date,soft,normal,hard
0,"데이터센터 전력 40% 차지하는 D램… 삼성·SK하이닉스, ‘전성비’ ...","챗GPT 시대 화두로 떠오른 전력효율성 문제 ”전력 먹는 하마, D램 전력효율성 개...",2023.07.10 15:29,"['산학협력', '소프트웨어적 호환성', '경쟁', 'D램 변환 계층', '삼성전자...","['소프트웨어적 호환성', 'D램 변환 계층', '삼성전자', 'HKMG 공정', ...","['전력효율성', 'SK하이닉스', 'D램', '삼성전자']"
1,"LS일렉트릭, 텍사스 공장 짓는다…현지 삼성·SK·LG 전력 지원",북미 첫 배전시스템 생산거점…생산설비 및 R&D센터 구축 삼성전자 파운드리 공장과 ...,2023.07.10 15:08,"['반도체', '전력인프라 부품', 'SK', 'LG에너지솔루션', '삼성전자', ...","['배스트럽시', '배터리', '반도체', 'LS일렉트릭', '전력인프라 부품', ...","['LS일렉트릭', 'R&D 센터', '배전시스템', '생산거점', '텍사스']"
2,“삼성전자가 식품도 팔았어?”…신규 가입 일단 종료한 사연,삼성 가전제품 구매고객에 삼성닷컴 내 e-식품관에서 할인혜택 주며 ‘락인’ 기대 ...,2023.07.10 15:07,"['e-식품관', '비스포크 가전', '온라인몰', 'LG씽큐 애플리케이션', '신...","['e-식품관', '비스포크 가전', '온라인몰', 'LG씽큐 애플리케이션', '삼...","['e-식품관', 'Neo QLED', '멤버십 플랜', '락인 전략', '신규 가..."
3,SGC솔루션 '도어글라스'…삼성·LG 세탁기·건조기에 공급,해외 가전 브랜드 공략…B2B 사업 확장 SGC솔루션 논산 공장. . 생활유리...,2023.07.10 15:05,"['세탁기', '유럽', '유리 제조 기술력', '도어 글라스', '대용량 드럼세탁...","['세탁기', '유럽', '유리 제조 기술력', '도어 글라스', '대용량 드럼세탁...","['세탁기', '도어 글라스', 'LG전자', '삼성전자', '건조기', 'SGC솔..."
4,‘페이커’ 내세운 삼성 OLED 게이밍 모니터 글로벌 3천대 돌파,북미·유럽 등 예약 판매 3000대 돌파 10일 오후 6시 삼성닷컴 ‘페이커’ 출연...,2023.07.10 14:58,"['삼성 OLED', '110 PPI 화소 밀도', '1800R 곡률의 커브드 스크...","['삼성 OLED', '110 PPI 화소 밀도', '1800R 곡률의 커브드 스크...","['삼성 OLED', '게이밍 모니터', '오디세이 OLED G9', '삼성전자',..."


### 테스트

### stopwords 설정 (기업명, 인물)

In [6]:
stopwords = '''
네이버
NAVER
Naver
naver
애플
LS일렉트릭
부강테크
블루오벌SK
배스트럽시
삼성
SAMSUNG
samsung
삼성전자
삼성그룹
삼성물산
삼성바이오
삼성벤처투자
SK
SK온
SK하이닉스
하이닉스
LG
LG전자
LG생활가전
LG에너지솔루션
KT
GS건설
마이크로소프트
쿠팡
엔비디아
제너럴모터스
SGC솔루션
카운터포인트리서치
포스코퓨처엠
평택화양
현대
현대모비스
IDC
TSMC
노태문
노태문 MX사업부장
팀쿡
팀 쿡
정훈
'''
stopwords = stopwords.split('\n')[1:-1]

In [41]:
stopwords

['네이버',
 'NAVER',
 'Naver',
 'naver',
 '애플',
 'LS일렉트릭',
 '부강테크',
 '블루오벌SK',
 '배스트럽시',
 '삼성',
 'SAMSUNG',
 'samsung',
 '삼성전자',
 '삼성그룹',
 '삼성물산',
 '삼성바이오',
 '삼성벤처투자',
 'SK',
 'SK온',
 'SK하이닉스',
 '하이닉스',
 'LG',
 'LG전자',
 'LG생활가전',
 'LG에너지솔루션',
 'KT',
 'GS건설',
 '마이크로소프트',
 '쿠팡',
 '엔비디아',
 '제너럴모터스',
 'SGC솔루션',
 '카운터포인트리서치',
 '포스코퓨처엠',
 '평택화양',
 '현대',
 '현대모비스',
 'IDC',
 'TSMC',
 '노태문',
 '노태문 MX사업부장',
 '팀쿡',
 '팀 쿡',
 '정훈']

### 설정 끝

## 모델 설계

In [18]:
import math

In [19]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [20]:
from konlpy.tag import Mecab, Okt

mecab = Mecab()
okt = Okt()

In [92]:
def get_words(text, stopwords = []):
    words = []
    for word in okt.phrases(text):
        if len(word.split()) == 1:
            if stopwords and (word in stopwords):
                continue
            words.append(word)

    return words

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [13]:
from sklearn.metrics.pairwise import cosine_similarity

In [21]:
from kss import split_sentences

### KR-WordRank

In [55]:
from kss import split_sentences
from krwordrank.word import KRWordRank

# Krwordrank 구현
class Zero:
    def __init__(self):
        pass
    
    def extract_keywords(self,
                         docs,
                         title,
                         top_k:int = 0,
                         beta = 0.85,
                         max_iter = 10,
                         stopwords = []
                         ):
        #키워드 후보 가져오기 가져오기
        wordrank_extractor = KRWordRank(min_count = 2, # 단어의 최소 출현 빈도수
                                        max_length = 6, # 단어의 최대 길이
                                        verbose = False
                                        )
        lines = [df['title'][0]] + split_sentences(docs)
        keywords, rank, graph = wordrank_extractor.extract(lines, beta, max_iter)

        if top_k:
            return [word for word, r in sorted(keywords.items(), key=lambda x:x[1], reverse=True) if word not in stopwords][:10]
        else:
            return [word for word, r in sorted(keywords.items(), key=lambda x:x[1], reverse=True) if word not in stopwords]


In [56]:
z=Zero()

In [64]:
z.extract_keywords(df['content'][0], df['title'][0], top_k=7, beta = 0.50, max_iter = 10, stopwords=stopwords)

['전력', 'D램', '있다.', '기술을', '전성비', '차세대', '위해', '설명했다.', '소비', '위한']

### 기본 자체 모델

In [13]:
# 기본 구조

class One:
    def __init__(self, model):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = SentenceTransformer(model).to(self.device)
    def extract_keywords(self,
                         docs,
                         top_k:int = 0,
                         max_count:int = 5,
                         stopwords = []
                         ):
        #키워드 후보 가져오기 가져오기
        words = get_words(docs, stopwords)

        #코사인 유사도
        doc_emb = self.model.encode(docs, device=self.device)
        word_emb = self.model.encode(words, device=self.device)

        text_word_cos = cosine_similarity([doc_emb], word_emb).flatten()

        #빈도수 점수
        word_counts = {word: docs.count(word) for word in words}
        frequency_score = []
        for count in word_counts.values():
            n = count if count < max_count else max_count
            frequency_score.append(math.log(n + 1, 100))
        frequency_score = np.array(frequency_score)

        #빈도수 점수 포함o
        all_keywords = []
        score = text_word_cos * frequency_score
        for i in np.argsort(-score):
            all_keywords.append((words[i], score[i]))
        if top_k:
            return [word for word, score in all_keywords][:top_k]
            # return [(word, score) for word, score in all_keywords][:top_k]
        else:
            return [word for word, score in all_keywords]
            # return [(word, score) for word, score in all_keywords]

        #빈도수 점수 포함x
        # all_keywords2 = []
        # for i in np.argsort(-text_word_cos):
        #     all_keywords2.append((words[i], text_word_cos[i]))
        # if top_k:
        #     return [word for word, score in all_keywords2][:top_k]
        #     # return [(word, score) for word, score in all_keywords][:top_k]
        # else:
        #     return [word for word, score in all_keywords2]
        #     # return [(word, score) for word, score in all_keywords]

In [14]:
# o = One('jhgan/ko-sroberta-multitask')
# o = One('jhgan/ko-sroberta-nli')
o = One('snunlp/KR-SBERT-V40K-klueNLI-augSTS')
# o4 = One('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')
# o5 = One('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

### 자체모델 + Sentence 위치에 따른 가중치

In [184]:
from konlpy.tag import Mecab
from kss import split_sentences


# sentence 위치에 따른 가중치
class Two:
    def __init__(self, model):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = SentenceTransformer(model).to(self.device)
        
    def extract_keywords(self,
                         docs,
                         title,
                         top_k:int = 0,
                         max_count:int = 5,
                         stopwords = []
                         ):
        #키워드 후보 가져오기 가져오기
        text = title + docs
        keyword_candidates = get_words(text, stopwords)

        #문장 분리와 타이틀 추가
        sentences = np.array([title] + split_sentences(docs))
        
        #키워드 임베딩
        keyword_embeddings = np.array([self.model.encode(x) for x in keyword_candidates])

        total_score = np.zeros(len(keyword_candidates))
        W = 2
        total_weight = 0
        for i, sentence in enumerate(sentences):
            sentence_embedding = self.model.encode(sentence)
            if i < 4: # or i >= n_sentences - 2:  # If it is one of the first two or last two sentences
                total_score += W * cosine_similarity(sentence_embedding.reshape(1, -1), keyword_embeddings)[0]
                total_weight += W
            # else:
                # total_score += 1 * cosine_similarity(sentence_embedding.reshape(1, -1), keyword_embeddings)[0]
                # total_weight += 1
        total_score = total_score/total_weight

        #빈도수 점수
        word_counts = {word: text.count(word) for word in keyword_candidates}
        frequency_score = []
        for count in word_counts.values():
            n = count if count < max_count else max_count
            frequency_score.append(math.log(n + 1, 100))
        frequency_score = np.array(frequency_score)

        #빈도수 점수 포함o
        all_keywords = []
        score = total_score * frequency_score
        for i in np.argsort(-score):
            all_keywords.append((keyword_candidates[i], score[i]))
        if top_k:
            # return [(word, score) for word, score in all_keywords][:top_k]
            return [word for word, score in all_keywords][:top_k]
        else:
            return [(word, score) for word, score in all_keywords]

In [185]:
t = Two("snunlp/KR-SBERT-V40K-klueNLI-augSTS")

### TextRank 방식 - 문장 가중치 설정

In [198]:
import networkx as nx
from konlpy.tag import Mecab
from kss import split_sentences

# TextRank 방식
class Three:
    def __init__(self, model):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = SentenceTransformer(model).to(self.device)
    
    def textrank(self, sentence_embeddings):
        sim_matrix = cosine_similarity(sentence_embeddings, sentence_embeddings)

        nx_graph = nx.from_numpy_array(sim_matrix)
        scores = nx.pagerank(nx_graph)
        return scores

    def extract_keywords(self,
                         docs,
                         title,
                         top_k:int = 0,
                         max_count:int = 5,
                         stopwords = []
                         ):
        #키워드 후보 가져오기 가져오기
        text = title + docs
        keyword_candidates = get_words(text, stopwords)

        #문장 분리와 타이틀 추가
        sentences = np.array([title] + split_sentences(docs))
        
        #키워드 임베딩
        keyword_embeddings = np.array([self.model.encode(x) for x in keyword_candidates])

        sentence_embeddings = [self.model.encode(x) for x in sentences]
        sentence_scores = self.textrank(sentence_embeddings)

        total_score = np.zeros(len(keyword_candidates))

        for i, sentence in enumerate(sentences):
            sentence_embedding = sentence_embeddings[i]
            weight = sentence_scores[i]
            total_score += weight * cosine_similarity(sentence_embedding.reshape(1, -1), keyword_embeddings)[0]

        #빈도수 점수
        word_counts = {word: text.count(word) for word in keyword_candidates}
        frequency_score = []
        for count in word_counts.values():
            n = count if count < max_count else max_count
            frequency_score.append(math.log(n + 1, 100))
        frequency_score = np.array(frequency_score)

        #빈도수 점수 포함o
        all_keywords = []
        score = total_score * frequency_score
        for i in np.argsort(-score):
            all_keywords.append((keyword_candidates[i], score[i]))
        if top_k:
            # return [(word, score) for word, score in all_keywords][:top_k]
            return [word for word, score in all_keywords][:top_k]
        else:
            return [(word, score) for word, score in all_keywords]

In [199]:
th = Three("snunlp/KR-SBERT-V40K-klueNLI-augSTS")

### MSS

In [217]:
import networkx as nx
from konlpy.tag import Mecab
from kss import split_sentences

# MSS 방식
class Four:
    def __init__(self, model):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = SentenceTransformer(model).to(self.device)
    
    def max_sum_sim(self, sentence_embeddings, sim_matrix, n):
        selected_index = []
        for i in range(n):
            sum_sim = []
            for i in range(len(sentence_embeddings)):
                if i in selected_index:  
                    sum_sim.append(0)
                    continue
                sum_sim.append(np.sum([sim_matrix[i][j] for j in selected_index]))
            idx_max = np.argmax(sum_sim)  
            selected_index.append(idx_max)
        return selected_index

    def extract_keywords(self,
                         docs,
                         title,
                         top_k:int = 0,
                         max_count:int = 5,
                         stopwords = []
                         ):
        #키워드 후보 가져오기 가져오기
        text = title + docs
        keyword_candidates = get_words(text, stopwords)

        #문장 분리와 타이틀 추가
        sentences = np.array([title] + split_sentences(docs))
        
        #키워드 임베딩
        keyword_embeddings = np.array([self.model.encode(x) for x in keyword_candidates])

        sentence_embeddings = [self.model.encode(x) for x in sentences]

        sim_matrix = cosine_similarity(sentence_embeddings, sentence_embeddings)
        selected_index = self.max_sum_sim(sentence_embeddings, sim_matrix, 4 if len(sentences) > 4 else 1)

        total_score = np.zeros(len(keyword_candidates))

        for i in selected_index:
            sentence_embedding = sentence_embeddings[i]
            total_score += cosine_similarity(sentence_embedding.reshape(1, -1), keyword_embeddings)[0]
        

        #빈도수 점수
        word_counts = {word: text.count(word) for word in keyword_candidates}
        frequency_score = []
        for count in word_counts.values():
            n = count if count < max_count else max_count
            frequency_score.append(math.log(n + 1, 100))
        frequency_score = np.array(frequency_score)

        #빈도수 점수 포함o
        all_keywords = []
        score = total_score * frequency_score
        for i in np.argsort(-score):
            all_keywords.append((keyword_candidates[i], score[i]))
        if top_k:
            # return [(word, score) for word, score in all_keywords][:top_k]
            return [word for word, score in all_keywords][:top_k]
        else:
            return [(word, score) for word, score in all_keywords]

In [218]:
f = Four("snunlp/KR-SBERT-V40K-klueNLI-augSTS")

### MMR 방식

In [94]:
import networkx as nx
from konlpy.tag import Mecab
from kss import split_sentences


# MMR 방식
class Five:
    def __init__(self, model):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = SentenceTransformer(model).to(self.device)
    
    def calculate_mmr_score(self, keyword_embedding, selected_keyword_embeddings, lambda_param=0.2):
        if len(selected_keyword_embeddings) == 0:  
            return 0
        keyword_similarity = cosine_similarity(keyword_embedding.reshape(1, -1), selected_keyword_embeddings)[0]
        return lambda_param * (1 - np.max(keyword_similarity))

    def extract_keywords(self,
                         docs,
                         title,
                         top_k:int = 0,
                         max_count:int = 5,
                         stopwords = []
                         ):
        #키워드 후보 가져오기 가져오기
        text = title + docs
        keyword_candidates = get_words(text, stopwords)

        #문장 분리와 타이틀 추가
        sentences = np.array([title] + split_sentences(docs))
        
        #키워드 임베딩
        keyword_embeddings = np.array([self.model.encode(x) for x in keyword_candidates])
        sentence_embeddings = [self.model.encode(x) for x in sentences]
        
        total_score = np.zeros(len(keyword_candidates))
        selected_keyword_embeddings = []
        for sentence_embedding in sentence_embeddings:
            similarity_scores = np.array([cosine_similarity(sentence_embedding.reshape(1, -1), keyword_embedding.reshape(1, -1))[0][0] for keyword_embedding in keyword_embeddings])
            mmr_scores = np.array([self.calculate_mmr_score(keyword_embedding, np.array(selected_keyword_embeddings)) for keyword_embedding in keyword_embeddings])
            total_score += similarity_scores - mmr_scores
            max_index = np.argmax(total_score)
            selected_keyword_embeddings.append(keyword_embeddings[max_index])

        #빈도수 점수
        word_counts = {word: text.count(word) for word in keyword_candidates}
        frequency_score = []
        for count in word_counts.values():
            n = count if count < max_count else max_count
            frequency_score.append(math.log(n + 1, 100))
        frequency_score = np.array(frequency_score)

        #빈도수 점수 포함o
        all_keywords = []
        score = total_score * frequency_score
        for i in np.argsort(-score):
            all_keywords.append((keyword_candidates[i], score[i]))
        if top_k:
            # return [(word, score) for word, score in all_keywords][:top_k]
            return [word for word, score in all_keywords][:top_k]
        else:
            return [(word, score) for word, score in all_keywords]

In [95]:
fi = Five("snunlp/KR-SBERT-V40K-klueNLI-augSTS")

In [96]:
df.head()

,title,content,date,soft,normal,hard
0,"데이터센터 전력 40% 차지하는 D램… 삼성·SK하이닉스, ‘전성비’ ...","챗GPT 시대 화두로 떠오른 전력효율성 문제 ”전력 먹는 하마, D램 전력효율성 개...",2023.07.10 15:29,"['산학협력', '소프트웨어적 호환성', '경쟁', 'D램 변환 계층', '삼성전자...","['소프트웨어적 호환성', 'D램 변환 계층', '삼성전자', 'HKMG 공정', ...","['전력효율성', 'SK하이닉스', 'D램', '삼성전자']"
1,"LS일렉트릭, 텍사스 공장 짓는다…현지 삼성·SK·LG 전력 지원",북미 첫 배전시스템 생산거점…생산설비 및 R&D센터 구축 삼성전자 파운드리 공장과 ...,2023.07.10 15:08,"['반도체', '전력인프라 부품', 'SK', 'LG에너지솔루션', '삼성전자', ...","['배스트럽시', '배터리', '반도체', 'LS일렉트릭', '전력인프라 부품', ...","['LS일렉트릭', 'R&D 센터', '배전시스템', '생산거점', '텍사스']"
2,“삼성전자가 식품도 팔았어?”…신규 가입 일단 종료한 사연,삼성 가전제품 구매고객에 삼성닷컴 내 e-식품관에서 할인혜택 주며 ‘락인’ 기대 ...,2023.07.10 15:07,"['e-식품관', '비스포크 가전', '온라인몰', 'LG씽큐 애플리케이션', '신...","['e-식품관', '비스포크 가전', '온라인몰', 'LG씽큐 애플리케이션', '삼...","['e-식품관', 'Neo QLED', '멤버십 플랜', '락인 전략', '신규 가..."
3,SGC솔루션 '도어글라스'…삼성·LG 세탁기·건조기에 공급,해외 가전 브랜드 공략…B2B 사업 확장 SGC솔루션 논산 공장. . 생활유리...,2023.07.10 15:05,"['세탁기', '유럽', '유리 제조 기술력', '도어 글라스', '대용량 드럼세탁...","['세탁기', '유럽', '유리 제조 기술력', '도어 글라스', '대용량 드럼세탁...","['세탁기', '도어 글라스', 'LG전자', '삼성전자', '건조기', 'SGC솔..."
4,‘페이커’ 내세운 삼성 OLED 게이밍 모니터 글로벌 3천대 돌파,북미·유럽 등 예약 판매 3000대 돌파 10일 오후 6시 삼성닷컴 ‘페이커’ 출연...,2023.07.10 14:58,"['삼성 OLED', '110 PPI 화소 밀도', '1800R 곡률의 커브드 스크...","['삼성 OLED', '110 PPI 화소 밀도', '1800R 곡률의 커브드 스크...","['삼성 OLED', '게이밍 모니터', '오디세이 OLED G9', '삼성전자',..."


### 테스트

In [75]:
n = 0
text = df['title'][n]+df['content'][n]
# check = o.extract_keywords(text, top_k = 5, stopwords = stopwords)
check = fi.extract_keywords(df['content'][n], df['title'][n], top_k = 7, stopwords = stopwords)

['전력', 'D램', '있다.', '기술을', '차세대', '전성비', '위해', '설명했다.', '문제', '소비']

In [98]:
check

['D램', '메모리', '소비전력', '전력효율성', 'DDR5', '효율성', '데이터센터']

In [97]:
df = pd.read_csv('42.csv')
df.head()

,title,content,date,soft,normal,hard
0,"데이터센터 전력 40% 차지하는 D램… 삼성·SK하이닉스, ‘전성비’ ...","챗GPT 시대 화두로 떠오른 전력효율성 문제 ”전력 먹는 하마, D램 전력효율성 개...",2023.07.10 15:29,"['산학협력', '소프트웨어적 호환성', '경쟁', 'D램 변환 계층', '삼성전자...","['소프트웨어적 호환성', 'D램 변환 계층', '삼성전자', 'HKMG 공정', ...","['전력효율성', 'SK하이닉스', 'D램', '삼성전자']"
1,"LS일렉트릭, 텍사스 공장 짓는다…현지 삼성·SK·LG 전력 지원",북미 첫 배전시스템 생산거점…생산설비 및 R&D센터 구축 삼성전자 파운드리 공장과 ...,2023.07.10 15:08,"['반도체', '전력인프라 부품', 'SK', 'LG에너지솔루션', '삼성전자', ...","['배스트럽시', '배터리', '반도체', 'LS일렉트릭', '전력인프라 부품', ...","['LS일렉트릭', 'R&D 센터', '배전시스템', '생산거점', '텍사스']"
2,“삼성전자가 식품도 팔았어?”…신규 가입 일단 종료한 사연,삼성 가전제품 구매고객에 삼성닷컴 내 e-식품관에서 할인혜택 주며 ‘락인’ 기대 ...,2023.07.10 15:07,"['e-식품관', '비스포크 가전', '온라인몰', 'LG씽큐 애플리케이션', '신...","['e-식품관', '비스포크 가전', '온라인몰', 'LG씽큐 애플리케이션', '삼...","['e-식품관', 'Neo QLED', '멤버십 플랜', '락인 전략', '신규 가..."
3,SGC솔루션 '도어글라스'…삼성·LG 세탁기·건조기에 공급,해외 가전 브랜드 공략…B2B 사업 확장 SGC솔루션 논산 공장. . 생활유리...,2023.07.10 15:05,"['세탁기', '유럽', '유리 제조 기술력', '도어 글라스', '대용량 드럼세탁...","['세탁기', '유럽', '유리 제조 기술력', '도어 글라스', '대용량 드럼세탁...","['세탁기', '도어 글라스', 'LG전자', '삼성전자', '건조기', 'SGC솔..."
4,‘페이커’ 내세운 삼성 OLED 게이밍 모니터 글로벌 3천대 돌파,북미·유럽 등 예약 판매 3000대 돌파 10일 오후 6시 삼성닷컴 ‘페이커’ 출연...,2023.07.10 14:58,"['삼성 OLED', '110 PPI 화소 밀도', '1800R 곡률의 커브드 스크...","['삼성 OLED', '110 PPI 화소 밀도', '1800R 곡률의 커브드 스크...","['삼성 OLED', '게이밍 모니터', '오디세이 OLED G9', '삼성전자',..."


In [77]:
df.head(2)

,title,content,date,soft,normal,hard
0,"데이터센터 전력 40% 차지하는 D램… 삼성·SK하이닉스, ‘전성비’ ...","챗GPT 시대 화두로 떠오른 전력효율성 문제 ”전력 먹는 하마, D램 전력효율성 개...",2023.07.10 15:29,"['산학협력', '소프트웨어적 호환성', '경쟁', 'D램 변환 계층', '삼성전자...","['소프트웨어적 호환성', 'D램 변환 계층', '삼성전자', 'HKMG 공정', ...","['전력효율성', 'SK하이닉스', 'D램', '삼성전자']"
1,"LS일렉트릭, 텍사스 공장 짓는다…현지 삼성·SK·LG 전력 지원",북미 첫 배전시스템 생산거점…생산설비 및 R&D센터 구축 삼성전자 파운드리 공장과 ...,2023.07.10 15:08,"['반도체', '전력인프라 부품', 'SK', 'LG에너지솔루션', '삼성전자', ...","['배스트럽시', '배터리', '반도체', 'LS일렉트릭', '전력인프라 부품', ...","['LS일렉트릭', 'R&D 센터', '배전시스템', '생산거점', '텍사스']"


### 모델 적용, 저장

In [98]:
# df['topk7'] = [o.extract_keywords(df['title'][n]+df['content'][n], top_k = 7, stopwords = stopwords) for n in range(df.shape[0])]
df['topk7'] = [z.extract_keywords(df['content'][n], df['title'][n], top_k = 7, stopwords = stopwords) for n in range(df.shape[0])]

In [99]:
df['topk5'] = df['topk7'].apply(lambda x: x[:5] if len(x)>=5 else x)
df['topk3'] = df['topk7'].apply(lambda x: x[:3] if len(x)>=3 else x)

In [80]:
df.head()

,title,content,date,soft,normal,hard,topk7,topk5,topk3
0,"데이터센터 전력 40% 차지하는 D램… 삼성·SK하이닉스, ‘전성비’ ...","챗GPT 시대 화두로 떠오른 전력효율성 문제 ”전력 먹는 하마, D램 전력효율성 개...",2023.07.10 15:29,"['산학협력', '소프트웨어적 호환성', '경쟁', 'D램 변환 계층', '삼성전자...","['소프트웨어적 호환성', 'D램 변환 계층', '삼성전자', 'HKMG 공정', ...","['전력효율성', 'SK하이닉스', 'D램', '삼성전자']","[전력, D램, 있다., 기술을, 차세대, 전성비, 위해, 설명했다., 문제, 소비]","[전력, D램, 있다., 기술을, 차세대]","[전력, D램, 있다.]"
1,"LS일렉트릭, 텍사스 공장 짓는다…현지 삼성·SK·LG 전력 지원",북미 첫 배전시스템 생산거점…생산설비 및 R&D센터 구축 삼성전자 파운드리 공장과 ...,2023.07.10 15:08,"['반도체', '전력인프라 부품', 'SK', 'LG에너지솔루션', '삼성전자', ...","['배스트럽시', '배터리', '반도체', 'LS일렉트릭', '전력인프라 부품', ...","['LS일렉트릭', 'R&D 센터', '배전시스템', '생산거점', '텍사스']","[중인, 있다., 배전시스템, 생산거점을, 통해, 매입한, LS일렉트릭은, 공장, ...","[중인, 있다., 배전시스템, 생산거점을, 통해]","[중인, 있다., 배전시스템]"
2,“삼성전자가 식품도 팔았어?”…신규 가입 일단 종료한 사연,삼성 가전제품 구매고객에 삼성닷컴 내 e-식품관에서 할인혜택 주며 ‘락인’ 기대 ...,2023.07.10 15:07,"['e-식품관', '비스포크 가전', '온라인몰', 'LG씽큐 애플리케이션', '신...","['e-식품관', '비스포크 가전', '온라인몰', 'LG씽큐 애플리케이션', '삼...","['e-식품관', 'Neo QLED', '멤버십 플랜', '락인 전략', '신규 가...","[플랜, 식품, 혜택을, 멤버십, 말했다., 최대, 이상, 신규, 판매, 구매]","[플랜, 식품, 혜택을, 멤버십, 말했다.]","[플랜, 식품, 혜택을]"
3,SGC솔루션 '도어글라스'…삼성·LG 세탁기·건조기에 공급,해외 가전 브랜드 공략…B2B 사업 확장 SGC솔루션 논산 공장. . 생활유리...,2023.07.10 15:05,"['세탁기', '유럽', '유리 제조 기술력', '도어 글라스', '대용량 드럼세탁...","['세탁기', '유럽', '유리 제조 기술력', '도어 글라스', '대용량 드럼세탁...","['세탁기', '도어 글라스', 'LG전자', '삼성전자', '건조기', 'SGC솔...","[도어, LG전자의, 국내, 삼성전자와, 대용량, 제조, SGC솔루션은, 가전, 사...","[도어, LG전자의, 국내, 삼성전자와, 대용량]","[도어, LG전자의, 국내]"
4,‘페이커’ 내세운 삼성 OLED 게이밍 모니터 글로벌 3천대 돌파,북미·유럽 등 예약 판매 3000대 돌파 10일 오후 6시 삼성닷컴 ‘페이커’ 출연...,2023.07.10 14:58,"['삼성 OLED', '110 PPI 화소 밀도', '1800R 곡률의 커브드 스크...","['삼성 OLED', '110 PPI 화소 밀도', '1800R 곡률의 커브드 스크...","['삼성 OLED', '게이밍 모니터', '오디세이 OLED G9', '삼성전자',...","[모니터, 게이밍, OLED, 판매, 오디세이, ‘페이커’가, 삼성전자는, 이상, ...","[모니터, 게이밍, OLED, 판매, 오디세이]","[모니터, 게이밍, OLED]"


In [100]:
df.to_csv('predict.csv',index=False)

In [26]:
df.head(2)

,title,content,date,soft,normal,hard,topk3,topk5,topk7
0,"데이터센터 전력 40% 차지하는 D램… 삼성·SK하이닉스, ‘전성비’ ...","챗GPT 시대 화두로 떠오른 전력효율성 문제 ”전력 먹는 하마, D램 전력효율성 개...",2023.07.10 15:29,"['산학협력', '소프트웨어적 호환성', '경쟁', 'D램 변환 계층', '삼성전자...","['소프트웨어적 호환성', 'D램 변환 계층', '삼성전자', 'HKMG 공정', ...","['전력효율성', 'SK하이닉스', 'D램', '삼성전자']","[D램, 메모리, 소비전력]","[D램, 메모리, 소비전력, 전력효율성, 효율성]","[D램, 메모리, 소비전력, 전력효율성, 효율성, DDR5, 차세대]"
1,"LS일렉트릭, 텍사스 공장 짓는다…현지 삼성·SK·LG 전력 지원",북미 첫 배전시스템 생산거점…생산설비 및 R&D센터 구축 삼성전자 파운드리 공장과 ...,2023.07.10 15:08,"['반도체', '전력인프라 부품', 'SK', 'LG에너지솔루션', '삼성전자', ...","['배스트럽시', '배터리', '반도체', 'LS일렉트릭', '전력인프라 부품', ...","['LS일렉트릭', 'R&D 센터', '배전시스템', '생산거점', '텍사스']","[LS, 생산거점, 공장]","[LS, 생산거점, 공장, 생산, 배전시스템]","[LS, 생산거점, 공장, 생산, 배전시스템, 일렉, 생산설비]"


## 평가

In [101]:
pred_df = pd.read_csv('predict.csv')

In [83]:
pred_df.head(2)

,title,content,date,soft,normal,hard,topk7,topk5,topk3
0,"데이터센터 전력 40% 차지하는 D램… 삼성·SK하이닉스, ‘전성비’ ...","챗GPT 시대 화두로 떠오른 전력효율성 문제 ”전력 먹는 하마, D램 전력효율성 개...",2023.07.10 15:29,"['산학협력', '소프트웨어적 호환성', '경쟁', 'D램 변환 계층', '삼성전자...","['소프트웨어적 호환성', 'D램 변환 계층', '삼성전자', 'HKMG 공정', ...","['전력효율성', 'SK하이닉스', 'D램', '삼성전자']","['전력', 'D램', '있다.', '기술을', '차세대', '전성비', '위해',...","['전력', 'D램', '있다.', '기술을', '차세대']","['전력', 'D램', '있다.']"
1,"LS일렉트릭, 텍사스 공장 짓는다…현지 삼성·SK·LG 전력 지원",북미 첫 배전시스템 생산거점…생산설비 및 R&D센터 구축 삼성전자 파운드리 공장과 ...,2023.07.10 15:08,"['반도체', '전력인프라 부품', 'SK', 'LG에너지솔루션', '삼성전자', ...","['배스트럽시', '배터리', '반도체', 'LS일렉트릭', '전력인프라 부품', ...","['LS일렉트릭', 'R&D 센터', '배전시스템', '생산거점', '텍사스']","['중인', '있다.', '배전시스템', '생산거점을', '통해', '매입한', '...","['중인', '있다.', '배전시스템', '생산거점을', '통해']","['중인', '있다.', '배전시스템']"


In [84]:
import ast

In [102]:
# list 모형의 str 을 list로 변경
for column in ['soft', 'normal', 'hard', 'topk3', 'topk5', 'topk7']:
    pred_df[column] = pred_df[column].apply(ast.literal_eval)

In [103]:
# soft, normal, hard 의 stopwords 제거
for column in ['soft', 'normal', 'hard']:
    pred_df[column] = pred_df[column].apply(lambda x: [word for word in x if word not in stopwords])

In [104]:
def compute_metrics(ground_truth, predictions):
    # Convert ground truth and predictions to sets
    ground_truth_set = set(ground_truth)
    predictions_set = set(predictions)

    # Calculate true positives
    tp = len(ground_truth_set & predictions_set)
    # Calculate false positives
    fp = len(predictions_set) - tp
    # Calculate false negatives
    fn = len(ground_truth_set) - tp

    # Calculate precision, recall, and F1 score
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f1

In [105]:
for level in ['soft', 'normal', 'hard']:
    for k in [3, 5, 7]:
        pred_df[f'{level}_topk{k}_precision'], pred_df[f'{level}_topk{k}_recall'], pred_df[f'{level}_topk{k}_f1'] = zip(*pred_df.apply(lambda row: compute_metrics(row[level], row[f'topk{k}']), axis=1))

In [107]:
metrics_summary = {}
for level in ['soft', 'normal', 'hard']:
    for k in [3, 5, 7]:
        metrics_summary[f'{level}_topk{k}_precision_avg'] = pred_df[f'{level}_topk{k}_precision'].mean()
        metrics_summary[f'{level}_topk{k}_recall_avg'] = pred_df[f'{level}_topk{k}_recall'].mean()
        metrics_summary[f'{level}_topk{k}_f1_avg'] = pred_df[f'{level}_topk{k}_f1'].mean()

In [108]:
metrics_summary

{'soft_topk3_precision_avg': 0.3095238095238095,
 'soft_topk3_recall_avg': 0.04496082321248519,
 'soft_topk3_f1_avg': 0.0774339480256633,
 'soft_topk5_precision_avg': 0.26666666666666666,
 'soft_topk5_recall_avg': 0.060350843772504784,
 'soft_topk5_f1_avg': 0.09674562647114746,
 'soft_topk7_precision_avg': 0.19761904761904764,
 'soft_topk7_recall_avg': 0.0869541778230921,
 'soft_topk7_f1_avg': 0.11825910241273171,
 'normal_topk3_precision_avg': 0.3095238095238095,
 'normal_topk3_recall_avg': 0.05780235347976632,
 'normal_topk3_f1_avg': 0.09581593581025145,
 'normal_topk5_precision_avg': 0.26190476190476186,
 'normal_topk5_recall_avg': 0.07590695072006247,
 'normal_topk5_f1_avg': 0.11534031990629245,
 'normal_topk7_precision_avg': 0.19285714285714284,
 'normal_topk7_recall_avg': 0.1116773578278203,
 'normal_topk7_f1_avg': 0.1383391276070874,
 'hard_topk3_precision_avg': 0.21428571428571425,
 'hard_topk3_recall_avg': 0.12552910052910055,
 'hard_topk3_f1_avg': 0.14941208806755024,
 'hard_